In this homework, we'll deploy the dino or dragon model we trained in the previous homework.

Download the model from here:

https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

import tensorflow.lite as tflite
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

### Question 1
Now convert this model from Keras to TF-Lite format.

What's the size of the __converted__ model?

- 21 Mb
- 43 Mb
- 80 Mb
- 164 Mb

In [2]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dino_dragon_10_0.899.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\mayan\AppData\Local\Temp\tmpb5111nnv\assets


INFO:tensorflow:Assets written to: C:\Users\mayan\AppData\Local\Temp\tmpb5111nnv\assets


In [4]:
# Get file size in bytes for a given model
os.stat('dino_dragon_10_0.899.tflite').st_size

44866124

__ANSWER__ : __b__ `43 Mb`

### Question 2
To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

- 3
- 7
- 13
- 24

In [5]:
interpreter = tflite.Interpreter(model_path='dino_dragon_10_0.899.tflite')
interpreter.allocate_tensors()

output_idx = interpreter.get_output_details()[0]['index']

In [6]:
output_idx

13

__ANSWER__ : __c__ `13 Mb`

### Preparing the image
You'll need some code for downloading and resizing images. You can use this code:

In [7]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg"
img = download_image(url)
img = prepare_image(img, target_size=(150,150))

### Question 3
Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

- 0.3353411
- 0.5529412
- 0.7458824
- 0.9654902

In [9]:
def prepare_input(x):
    return x / 255.0

In [10]:
x = np.array(img, dtype='float32')
X = preprocess_input(x)
X = np.array([X])
X = prepare_input(X)
X[0][0][0][0]

0.00041522513

In [11]:
X.shape

(1, 150, 150, 3)

__ANSWER__ : __b__ `0.5529412`

### Question 4
Now let's apply this model to this image. What's the output of the model?

- 0.17049132
- 0.39009996
- 0.60146114
- 0.82448614

In [12]:
input_idx = interpreter.get_input_details()[0]['index']
interpreter.set_tensor(input_idx, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_idx)
preds

array([[0.22139649]], dtype=float32)

__ANSWER__ : __d__ `0.82448614`

### Docker
For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:

FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .
And pushed it to svizor42/zoomcamp-dino-dragon-lambda:v2.

A few notes:

The image already contains a model and it's not the same model as the one we used for questions 1-4.
The version of Python is 3.9, so you need to use the right wheel for TF-Lite. For Tensorflow 2.7.0, it's https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.7.0-cp39-cp39-linux_x86_64.whl

### Question 5
Download the base image svizor42/zoomcamp-dino-dragon-lambda:v2. You can easily make it by using docker pull command.

So what's the size of this base image?

- 139 Mb
- 329 Mb
- 639 Mb
- 929 Mb

You can get this information when running docker images - it'll be in the "SIZE" column.

In [13]:
!docker images

REPOSITORY                             TAG           IMAGE ID       CREATED       SIZE
svizor42/zoomcamp-dino-dragon-lambda   v2            20ef58b21a05   11 days ago   639MB
app                                    latest        867c1c61a59c   3 weeks ago   1.71GB
mlapp                                  latest        28c771ba5243   3 weeks ago   1.67GB
flights                                latest        1f8db05c37a2   3 weeks ago   1.88GB
<none>                                 <none>        b27ab44823bf   3 weeks ago   1.88GB
<none>                                 <none>        423442317309   3 weeks ago   1.88GB
docker_tutorial                        latest        fd0f462016a3   3 weeks ago   250MB
<none>                                 <none>        7111a8626832   3 weeks ago   211MB
hello                                  latest        ce89620bd7ea   7 weeks ago   585MB
<none>                                 <none>        ff0d39b77581   7 weeks ago   585MB
<none>                      

__ANSWER__ : __c__ `639MB`

### Question 6

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is dino-vs-dragon-v2.tflite and it's in the current workdir in the image (see the Dockerfile above for the reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

What's the output from the model?

- 0.12
- 0.32
- 0.52
- 0.72

In [14]:
url = "https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg"
img = download_image(url)
img = prepare_image(img, target_size=(150,150))

x = np.array(img, dtype='float32')
X = preprocess_input(X)
X = np.array([x])
X = prepare_input(X)

interpreter.set_tensor(input_idx, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_idx)
preds

array([[0.3495881]], dtype=float32)

__ANSWER__ : __b__ `0.32`